In [1]:
import openai
from openai import OpenAI
import os
import time
import pandas as pd
from dotenv import load_dotenv 

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY') # For replication, either create an .env file and insert your OPENAI_API_KEY there or directly replace api key here
client = OpenAI()

In [2]:
# IF YOU CHOOSE TO RUN THIS SCRIPT INDIVIDUALLY, REPLACE "~/Desktop/Univ. of Chicago/AI Judges/" WITH THE ACTUAL FILE PATH WHERE THE REPLICATION PACKAGE IS STORED ON YOUR COMPUTER
user_file_path = os.path.expanduser("~/Desktop/Univ. of Chicago/AI Judges/")

In [8]:
# Directory where the text files are located
directory = os.path.join(user_file_path, "Replication Package", "Experiment Materials", "Input Materials")

# Dictionary containing lists of file paths
files_dict = {
    "instructions": [
        "Prompt Engineering/Instructions, Briefs, & SOAF/Horvat_Panel_Instructions.txt",
        "Prompt Engineering/Instructions, Briefs, & SOAF/Vukovic_Panel_Instructions.txt",
        "Prompt Engineering/Instructions, Briefs, & SOAF/Horvat_Sympathy1_Instructions.txt",
        "Prompt Engineering/Instructions, Briefs, & SOAF/Horvat_Sympathy2_Instructions.txt",
        "Prompt Engineering/Instructions, Briefs, & SOAF/Horvat_Sympathy3_Instructions.txt",
        "Prompt Engineering/Instructions, Briefs, & SOAF/Vukovic_Sympathy1_Instructions.txt",
        "Prompt Engineering/Instructions, Briefs, & SOAF/Vukovic_Sympathy2_Instructions.txt",
        "Prompt Engineering/Instructions, Briefs, & SOAF/Vukovic_Sympathy3_Instructions.txt",
        "Prompt Engineering/Instructions, Briefs, & SOAF/Horvat_Fuller_Instructions.txt",
        "Prompt Engineering/Instructions, Briefs, & SOAF/Formalism_Instructions.txt",
        "Prompt Engineering/Instructions, Briefs, & SOAF/Legal_Positivism_Instructions.txt",
        "Prompt Engineering/Instructions, Briefs, & SOAF/Legal_Pragmatism_Instructions.txt",
        "Prompt Engineering/Instructions, Briefs, & SOAF/Legal_Realism_Instructions.txt",
        "Prompt Engineering/Instructions, Briefs, & SOAF/Natural_Law_Instructions.txt", 
        "Prompt Engineering/Instructions, Briefs, & SOAF/Strict_Legal_Positivism_Instructions.txt",
        "Prompt Engineering/Instructions, Briefs, & SOAF/Horvat_Evaluate_Instructions.txt",
        "Prompt Engineering/Instructions, Briefs, & SOAF/Vukovic_Evaluate_Instructions.txt",
        "Prompt Engineering/Instructions, Briefs, & SOAF/Horvat_Experiment_Instructions.txt",
        "Prompt Engineering/Instructions, Briefs, & SOAF/Vukovic_Experiment_Instructions.txt"
    ],
    "final_lines": [
        "Prompt Engineering/Final Line/Panel_Final_Line.txt",
        "Prompt Engineering/Final Line/Sympathy1_Final_Line.txt",
        "Prompt Engineering/Final Line/Sympathy2_Final_Line.txt",
        "Prompt Engineering/Final Line/Sympathy3_Final_Line.txt",
        "Prompt Engineering/Final Line/Evaluate_Final_Line.txt",
        "Prompt Engineering/Final Line/Experiment_Final_Line.txt"
    ],
    "system_prompts": [
        "Prompt Engineering/System Prompt/Panel_System_Prompt.txt",
        "Prompt Engineering/System Prompt/Original_System_Prompt.txt",
        "Prompt Engineering/System Prompt/Fuller_System_Prompt.txt",
        "Prompt Engineering/System Prompt/Evaluate_System_Prompt.txt",
        "Prompt Engineering/System Prompt/Experiment_System_Prompt.txt",
    ],
    # Briefs and statement of agreed facts are in the same .txt files
    "briefs": [
        "Instructions, Briefs, & SOAF/Horvat_Sainovic.txt",
        "Instructions, Briefs, & SOAF/Horvat_Vasiljevic.txt",
        "Instructions, Briefs, & SOAF/Vukovic_Sainovic.txt",
        "Instructions, Briefs, & SOAF/Vukovic_Vasiljevic.txt"
    ],
    "statute": [
        "Cases & Statute/Statute.txt",
    ],
    "precedents": [
        "Cases & Statute/Appeal_Sainovic.txt", 
        "Cases & Statute/Appeal_Vasiljevic.txt"
    ], 
    "trials": [
        "Cases & Statute/Trial_Horvat_1.txt",
        "Cases & Statute/Trial_Horvat_2.txt",
        "Cases & Statute/Trial_Vukovic_1.txt",
        "Cases & Statute/Trial_Vukovic_2.txt"
    ]
}

# Function to read text from files
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Loop through the dictionary to read and store each text file with the original variable names
for category, file_list in files_dict.items():
    for file_name in file_list:
        file_path = os.path.join(directory, file_name)
        # Clean up the filename to create the variable name (removes spaces and extensions)
        key = os.path.splitext(os.path.basename(file_name))[0].replace(" ", "_")
        globals()[key] = read_text_file(file_path)

In [9]:
# Assigning model specifications (change as needed)
temperature = .7
model = 'gpt-4o-2024-05-13'

In [10]:
# Defining prompt and functions to summarize longer cases (precedent & trial judgment)
summarize_precedent_prompt = """Please summarize the following court case. Include an overview of the charges, the prosecution's and defense's main arguments, and the court's findings. Emphasize the reasoning behind the court’s decision and any notable legal interpretations. Ensure the summary captures the essence of the court’s conclusions and that it is thoroughly comprehensive.
Here is the case: """

summarize_trial1_prompt = """Please summarize the following court case. Include an overview of the charges, the prosecution's and defense's main arguments, and any other relevant legal information. Please note that because the case in its entirety exceeds GPT 4's context limit, you will only be provided with the first half of the case. Ensure the summary captures the essence of what the case is about and the legal issues it entails. Make sure your summary is thoroughly comprehensive.
Here is the first half of the trial judgment: """ 

def summarize_precedent(Precedent, seed):
        precedent_response = client.chat.completions.create(
            model=model,
            temperature=temperature,
            seed=seed,
            messages=[
            {"role": "system", "content": "You are an appeals judge in a pending case at the International Criminal Tribunal for the Former Yugoslavia (ICTY)."},
            {"role": "user", "content": summarize_precedent_prompt + Precedent}
        ]
        )
        return precedent_response.choices[0].message.content

def summarize_trial1(First_Half, seed):
        trial1_response = client.chat.completions.create(
            model=model,
            temperature=temperature,
            seed=seed,
            messages=[
            {"role": "system", "content": "You are an appeals judge in a pending case at the International Criminal Tribunal for the Former Yugoslavia (ICTY)."},
            {"role": "user", "content": summarize_trial1_prompt + First_Half}
        ]   
        )
        return trial1_response.choices[0].message.content

def summarize_trial2(trial1_summary, Second_Half, seed):
        summarize_trial2_prompt = f"""Please summarize the following court case. Note that this is not the full case; it is only the second half. This is due to the full case exceeding GPT 4's context limit. For reference, here is a summary of the first half of the case:{trial1_summary} 
        Now that you have a summary of the first half on hand, please summarize the remainder of the case. Include any relevant legal information. Emphasize the reasoning behind the court’s decision and any notable legal interpretations. Ensure the summary captures the essence of the court’s conclusions and that it is thoroughly comprehensive.
        Here is the second half of the trial judgment: """    
        trial2_response = client.chat.completions.create(
            model=model,
            temperature=temperature,
            seed=seed,
            messages=[
            {"role": "system", "content": "You are an appeals judge in a pending case at the International Criminal Tribunal for the Former Yugoslavia (ICTY)."},
            {"role": "user", "content": summarize_trial2_prompt + Second_Half}
        ]   
        )    
        return trial2_response.choices[0].message.content  

In [11]:
def simulate_judicial_decision(Instructions, Condition, Condition_Name, Statute, Precedent, First_Half, Second_Half, final_line, system_prompt, prompt_type, seed):

    precedent_summary = summarize_precedent(Precedent, seed)
    time.sleep(10)
    trial1_summary = summarize_trial1(First_Half, seed)
    time.sleep(10)
    trial2_summary = summarize_trial2(trial1_summary, Second_Half, seed)
    time.sleep(10)

    final_input = f"""{Instructions}

    Here is the Statement of Agreed Facts, the Brief for the Defense, and the Brief for the Prosecution: 

    {Condition}

    Here is the Statute: 

    {Statute} 

    Here is the summary for the Precedent: 

    {precedent_summary} 

    Here is the summary for the first half of the Trial Judgment: 

    {trial1_summary}

    Here is the summary for the second half of the Trial Judgment: 

    {trial2_summary}
    
    {final_line}
    """

    response = client.chat.completions.create(
        model=model,
        temperature=temperature,
        seed=seed,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": final_input}
        ]  
    )
    final_decision = response.choices[0].message.content 

    output_directory = os.path.join(user_file_path, "Replication Package", "Results", "Prompt Engineering", f"GPT Results ({prompt_type})")
    os.makedirs(output_directory, exist_ok=True)
    output_file_name = f"{Condition_Name}_{seed}.txt"
    path = os.path.join(output_directory, output_file_name)

    with open(path, "w") as file:
        file.write("Final Decision:\n") 
        file.write(final_decision)


In [1]:
import random
random.seed(25)

seeds = [random.randint(0, 1000000) for _ in range(25)]
first_seed = seeds[:1]

[395273, 805816, 971808, 884512, 884418, 16083, 224443, 976196, 914052, 319555, 667378, 496575, 44451, 899723, 999263, 794174, 268103, 36406, 320760, 592170, 444680, 100821, 617101, 900339, 130434]
[395273]


In [11]:
# CODE TO RUN PANEL OF JUDGES PROMPT

for seed in seeds:
    simulate_judicial_decision(Horvat_Panel_Instructions, Horvat_Sainovic, "Horvat_Sainovic", Statute, Appeal_Sainovic, Trial_Horvat_1, Trial_Horvat_2, Panel_Final_Line, Panel_System_Prompt, "Panel", seed=seed)
    simulate_judicial_decision(Horvat_Panel_Instructions, Horvat_Vasiljevic, "Horvat_Vasiljevic", Statute, Appeal_Vasiljevic, Trial_Horvat_1, Trial_Horvat_2, Panel_Final_Line, Panel_System_Prompt, "Panel", seed=seed)
    simulate_judicial_decision(Vukovic_Panel_Instructions, Vukovic_Sainovic, "Vukovic_Sainovic", Statute, Appeal_Sainovic, Trial_Vukovic_1, Trial_Vukovic_2, Panel_Final_Line, Panel_System_Prompt, "Panel", seed=seed)
    simulate_judicial_decision(Vukovic_Panel_Instructions, Vukovic_Vasiljevic, "Vukovic_Vasiljevic", Statute, Appeal_Vasiljevic, Trial_Vukovic_1, Trial_Vukovic_2, Panel_Final_Line, Panel_System_Prompt, "Panel", seed=seed)

In [ ]:
# CODE TO RUN SYMPATH (VARIATION 1) PROMPT

for seed in seeds:
    simulate_judicial_decision(Horvat_Sympathy1_Instructions, Horvat_Sainovic, "Horvat_Sainovic", Statute, Appeal_Sainovic, Trial_Horvat_1, Trial_Horvat_2, Sympathy1_Final_Line, Original_System_Prompt, "Sympathy1", seed=seed)
    simulate_judicial_decision(Horvat_Sympathy1_Instructions, Horvat_Vasiljevic, "Horvat_Vasiljevic", Statute, Appeal_Vasiljevic, Trial_Horvat_1, Trial_Horvat_2, Sympathy1_Final_Line, Original_System_Prompt, "Sympathy1", seed=seed)
    simulate_judicial_decision(Vukovic_Sympathy1_Instructions, Vukovic_Sainovic, "Vukovic_Sainovic", Statute, Appeal_Sainovic, Trial_Vukovic_1, Trial_Vukovic_2, Sympathy1_Final_Line, Original_System_Prompt, "Sympathy1", seed=seed)
    simulate_judicial_decision(Vukovic_Sympathy1_Instructions, Vukovic_Vasiljevic, "Vukovic_Vasiljevic", Statute, Appeal_Vasiljevic, Trial_Vukovic_1, Trial_Vukovic_2, Sympathy1_Final_Line, Original_System_Prompt, "Sympathy1", seed=seed)

In [ ]:
# CODE TO RUN SYMPATH (VARIATION 2) PROMPT

for seed in seeds:
    simulate_judicial_decision(Horvat_Sympathy2_Instructions, Horvat_Sainovic, "Horvat_Sainovic", Statute, Appeal_Sainovic, Trial_Horvat_1, Trial_Horvat_2, Sympathy2_Final_Line, Original_System_Prompt, "Sympathy2", seed=seed)
    simulate_judicial_decision(Horvat_Sympathy2_Instructions, Horvat_Vasiljevic, "Horvat_Vasiljevic", Statute, Appeal_Vasiljevic, Trial_Horvat_1, Trial_Horvat_2, Sympathy2_Final_Line, Original_System_Prompt, "Sympathy2", seed=seed)
    simulate_judicial_decision(Vukovic_Sympathy2_Instructions, Vukovic_Sainovic, "Vukovic_Sainovic", Statute, Appeal_Sainovic, Trial_Vukovic_1, Trial_Vukovic_2, Sympathy2_Final_Line, Original_System_Prompt, "Sympathy2", seed=seed)
    simulate_judicial_decision(Vukovic_Sympathy2_Instructions, Vukovic_Vasiljevic, "Vukovic_Vasiljevic", Statute, Appeal_Vasiljevic, Trial_Vukovic_1, Trial_Vukovic_2, Sympathy2_Final_Line, Original_System_Prompt, "Sympathy2", seed=seed)

In [ ]:
# CODE TO RUN SYMPATH (VARIATION 3) PROMPT

for seed in seeds:
    simulate_judicial_decision(Horvat_Sympathy3_Instructions, Horvat_Sainovic, "Horvat_Sainovic", Statute, Appeal_Sainovic, Trial_Horvat_1, Trial_Horvat_2, Sympathy3_Final_Line, Original_System_Prompt, "Sympathy3", seed=seed)
    simulate_judicial_decision(Horvat_Sympathy3_Instructions, Horvat_Vasiljevic, "Horvat_Vasiljevic", Statute, Appeal_Vasiljevic, Trial_Horvat_1, Trial_Horvat_2, Sympathy3_Final_Line, Original_System_Prompt, "Sympathy3", seed=seed)
    simulate_judicial_decision(Vukovic_Sympathy3_Instructions, Vukovic_Sainovic, "Vukovic_Sainovic", Statute, Appeal_Sainovic, Trial_Vukovic_1, Trial_Vukovic_2, Sympathy3_Final_Line, Original_System_Prompt, "Sympathy3", seed=seed)
    simulate_judicial_decision(Vukovic_Sympathy3_Instructions, Vukovic_Vasiljevic, "Vukovic_Vasiljevic", Statute, Appeal_Vasiljevic, Trial_Vukovic_1, Trial_Vukovic_2, Sympathy3_Final_Line, Original_System_Prompt, "Sympathy3", seed=seed)

In [ ]:
# CODE TO RUN EVALUATE THE LOWER COURT PROMPT

for seed in seeds:
    simulate_judicial_decision(Horvat_Evaluate_Instructions, Horvat_Sainovic, "Horvat_Sainovic", Statute, Appeal_Sainovic, Trial_Horvat_1, Trial_Horvat_2, Evaluate_Final_Line, Evaluate_System_Prompt, "Evaluate", seed=seed)
    simulate_judicial_decision(Horvat_Evaluate_Instructions, Horvat_Vasiljevic, "Horvat_Vasiljevic", Statute, Appeal_Vasiljevic, Trial_Horvat_1, Trial_Horvat_2, Evaluate_Final_Line, Evaluate_System_Prompt, "Evaluate", seed=seed)
    simulate_judicial_decision(Vukovic_Evaluate_Instructions, Vukovic_Sainovic, "Vukovic_Sainovic", Statute, Appeal_Sainovic, Trial_Vukovic_1, Trial_Vukovic_2, Evaluate_Final_Line, Evaluate_System_Prompt, "Evaluate", seed=seed)
    simulate_judicial_decision(Vukovic_Evaluate_Instructions, Vukovic_Vasiljevic, "Vukovic_Vasiljevic", Statute, Appeal_Vasiljevic, Trial_Vukovic_1, Trial_Vukovic_2, Evaluate_Final_Line, Evaluate_System_Prompt, "Evaluate", seed=seed)

In [ ]:
# CODE TO RUN SOCIAL SCIENCE EXPERIMENT PROMPT

for seed in seeds:
    simulate_judicial_decision(Horvat_Experiment_Instructions, Horvat_Sainovic, "Horvat_Sainovic", Statute, Appeal_Sainovic, Trial_Horvat_1, Trial_Horvat_2, Experiment_Final_Line, Experiment_System_Prompt, "Experiment", seed=seed)
    simulate_judicial_decision(Horvat_Experiment_Instructions, Horvat_Vasiljevic, "Horvat_Vasiljevic", Statute, Appeal_Vasiljevic, Trial_Horvat_1, Trial_Horvat_2, Experiment_Final_Line, Experiment_System_Prompt, "Experiment", seed=seed)
    simulate_judicial_decision(Vukovic_Experiment_Instructions, Vukovic_Sainovic, "Vukovic_Sainovic", Statute, Appeal_Sainovic, Trial_Vukovic_1, Trial_Vukovic_2, Experiment_Final_Line, Experiment_System_Prompt, "Experiment", seed=seed)
    simulate_judicial_decision(Vukovic_Experiment_Instructions, Vukovic_Vasiljevic, "Vukovic_Vasiljevic", Statute, Appeal_Vasiljevic, Trial_Vukovic_1, Trial_Vukovic_2, Experiment_Final_Line, Experiment_System_Prompt, "Experiment", seed=seed)

In [ ]:
# CODE TO RUN FULLER PROMPT (ONLY ON SYMPATHETIC/P-AFFIRM CONDITION)
# Note this only ones run iteration

for seed in first_seed:
    simulate_judicial_decision(Horvat_Fuller_Instructions, Horvat_Sainovic, "Horvat_Sainovic", Statute, Appeal_Sainovic, Trial_Horvat_1, Trial_Horvat_2, Fuller_Final_Line, Fuller_System_Prompt, "Fuller", seed=seed)

In [ ]:
# CODE TO RUN INDIVIDUAL PHILOSPHIES; requires new script because final line is dynamic
# Note this only ones run iteration

def simulate_judicial_decision_individual_philosophies(Instructions, Condition, Condition_Name, Statute, Precedent, First_Half, Second_Half, judicial_philosophy, seed):

    precedent_summary = summarize_precedent(Precedent, seed)
    time.sleep(10)
    trial1_summary = summarize_trial1(First_Half, seed)
    time.sleep(10)
    trial2_summary = summarize_trial2(trial1_summary, Second_Half, seed)
    time.sleep(10)

    final_input = f"""{Instructions}

    Here is the Statement of Agreed Facts, the Brief for the Defense, and the Brief for the Prosecution: 

    {Condition}

    Here is the Statute: 

    {Statute} 

    Here is the summary for the Precedent: 

    {precedent_summary} 

    Here is the summary for the first half of the Trial Judgment: 

    {trial1_summary}

    Here is the summary for the second half of the Trial Judgment: 

    {trial2_summary}
    
    Now that you have read all of the relevant information, please provide a decision on whether to affirm the lower court’s decision or not (indicate this by either saying “Affirm” or “Reverse”), as well as a brief 1-paragraph description of your rationale.
    Make sure to include in your rationale why your decision aligns with a {judicial_philosophy} approach.
    """

    response = client.chat.completions.create(
        model=model,
        temperature=temperature,
        seed=seed,
        messages=[
            {"role": "system", "content": f"You are analyzing a pending case at the International Criminal Tribunal for the Former Yugoslavia (ICTY). Your task is to determine whether various judges would affirm or reverse the lower court's decision, using a {judicial_philosophy} approach."},
            {"role": "user", "content": final_input}
        ]  
    )
    final_decision = response.choices[0].message.content 

    output_directory = os.path.join(user_file_path, "Replication Package", "Results", "Prompt Engineering", "GPT Results (Individual Philosophies)")
    os.makedirs(output_directory, exist_ok=True)
    output_file_name = f"Horvat_Sainovic_{judicial_philosophy}_{seed}.txt"
    path = os.path.join(output_directory, output_file_name)

    with open(path, "w") as file:
        file.write("Final Decision:\n") 
        file.write(final_decision)

    return final_decision

for seed in first_seed:
    simulate_judicial_decision(Legal_Positivism_Instructions, Horvat_Sainovic, "Horvat_Sainovic", Statute, Appeal_Sainovic, Trial_Horvat_1, Trial_Horvat_2, "Legal Positivism", seed=seed)
    simulate_judicial_decision(Natural_Law_Instructions, Horvat_Sainovic, "Horvat_Sainovic", Statute, Appeal_Sainovic, Trial_Horvat_1, Trial_Horvat_2, "Natural Law", seed=seed)
    simulate_judicial_decision(Legal_Realism_Instructions, Horvat_Sainovic, "Horvat_Sainovic", Statute, Appeal_Sainovic, Trial_Horvat_1, Trial_Horvat_2, "Legal Realism", seed=seed)
    simulate_judicial_decision(Strict_Legal_Positivism_Instructions, Horvat_Sainovic, "Horvat_Sainovic", Statute, Appeal_Sainovic, Trial_Horvat_1, Trial_Horvat_2, "Strict Legal Positivism", seed=seed)
    simulate_judicial_decision(Legal_Pragmatism_Instructions, Horvat_Sainovic, "Horvat_Sainovic", Statute, Appeal_Sainovic, Trial_Horvat_1, Trial_Horvat_2, "Legal Pragmatism", seed=seed)
    simulate_judicial_decision(Formalism_Instructions, Horvat_Sainovic, "Horvat_Sainovic", Statute, Appeal_Sainovic, Trial_Horvat_1, Trial_Horvat_2, "Legal Formalism", seed=seed)